## Plotting Installation:

The plotting package allows you to make an interactive CRN plot. Plotting requires the [Bokeh](https://docs.bokeh.org/en/latest/docs/installation.html) and [ForceAtlas2](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0098679) libraries to be installed on your machine. Bokeh is used for plotting, ForceAtlas2 is used for graph layout. To install, type the following into the consol:
> Conda install bokeh \\
pip install fa2

Alternatively, follow the installation instructions in the links above.




## Plotting Example
The CRN plot has a default representation:
* species are circles
    - orange circles are RNA species
    - blue circles are complexes
    - green circles are proteins
    - grey circles are DNA
    -there's always a purple circle that represents nothing. If your CRN doesn't have reactions that go to nothing, then it won't have any connections.
    
* reactions are squares

Click on a node (either reaction or species) and all arrows including that node will be highlighted
Mouse over a node (either reaction or species) and a tooltip will tell you what it is.

In [1]:
%matplotlib inline

from biocrnpyler import *
import numpy as np

txtl = CRNLab("GFP")
txtl.mixture("mixture1", extract = "TxTlExtract", mixture_volume = 1e-6, mixture_parameters = 'BasicExtract.tsv')
dna = DNAassembly("mydna",promoter=RegulatedPromoter("plac",["laci"]),rbs="UTR1",protein="GFP")
txtl.add_dna(name = "G1", promoter = "pBest", rbs = "BCD2", protein = "GFP", initial_conc = 10, volume = 1e-7)
well1 = txtl.get_model()

In [2]:
from bokeh.models import (Plot , Range1d)
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook() #this makes the graph appear in line with the notebook
DG, DGspec, DGrxn = generate_networkx_graph(well1) #this creates the networkx objects
plot = Plot(plot_width=500, plot_height=500, x_range=Range1d(-500, 500), y_range=Range1d(-500, 500)) #this generates a bokeh plot
graphPlot(DG,DGspec,DGrxn,plot,layout="force",posscale=1) #now you draw the network on the plot. Layout "force" is the default. 
#"posscale" scales the entire graph. This mostly just affects the size of the arrows relative to everything else
bokeh.io.show(plot) #if you don't type this the plot won't show

#mouse over nodes to get a tooltip telling you what they are
#mouse over single lines to outline them
#click on nodes to highlight all edges that touch them
#use the magnifying glass symbol to zoom in and out
#click and drag to move around

#NOTE: this function is not deterministic in how it displays the network, because it uses randomness to push the nodes around

Loading BokehJS ...

D:\anaconda3\lib\site-packages\bokeh\models\graphs.py:164: UserWarning: Node keys in 'layout_function' don't match node keys in the graph. These nodes may not be displayed correctly.
  warn("Node keys in 'layout_function' don't match node keys in the graph. "


In [5]:
#this demonstrates the "circle" layout. reactions are in the middle with species on the outside.
#also, the pretty_print text display style
DG, DGspec, DGrxn = generate_networkx_graph(well1,use_pretty_print=True,pp_show_rates=False,pp_show_attributes=False) 
plot = Plot(plot_width=500, plot_height=500, x_range=Range1d(-500, 500), y_range=Range1d(-500, 500)) 
graphPlot(DG,DGspec,DGrxn,plot,layout="circle",posscale=1) 
bokeh.io.show(plot) 